In [55]:
#!conda install -c conda-forge lightgbm
#!pip install optuna

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import sys
import os
import re
import random

from time import time
from tqdm import tqdm

from contextlib import contextmanager
import lightgbm as lgb
#import optuna.integration.lightgbm as opt_lgb

In [58]:
#上限表示数を拡張
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 200)

In [59]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [60]:
INPUT = "/content/drive/MyDrive/nishika/" # 所望のディレクトリに変更してください。
train_df = pd.read_csv(os.path.join(INPUT, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT, "test.csv"))
submission_df = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))

In [61]:
#df_1_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_SwinTransformerLarge384_V2Large256.csv")
#df_1_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_SwinTransformerLarge384_V2Large256.csv")

In [62]:
df_2_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_efb2_rinnabase.csv")
df_2_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_efb2_rinnabase.csv")

In [63]:
df_2_tr = df_2_tr.drop(df_2_tr.columns[352:], axis=1)

In [64]:
df_2_ts = df_2_ts.drop(df_2_ts.columns[352:], axis=1)

In [65]:
#df_3_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_swinv2base256_ginza510electra.csv")
#df_3_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_swinv2base256_ginza510electra.csv")

In [66]:
df_4_tr = pd.read_csv("/content/drive/MyDrive/nishika/output/train_efv2b2_resnet152_jaginza.csv")
df_4_ts = pd.read_csv("/content/drive/MyDrive/nishika/output/test_efv2b2_resnet152_jaginza.csv")

In [67]:
df_4_tr = df_4_tr.drop(df_4_tr.columns[352:], axis=1)

In [68]:
df_4_ts = df_4_ts.drop(df_4_ts.columns[352:], axis=1)

In [69]:
#swinL384,swinV2L256,Roberta-text,efb2
#df_tmp_tr_1 = pd.concat([df_1_tr,df_2_tr],axis=1)
#df_tmp_ts_1 = pd.concat([df_1_ts,df_2_ts],axis=1)

In [70]:
#ginza510-electra-base-text,swinv2base256
#df_tmp_tr_2 = pd.concat([df_tmp_tr_1,df_3_tr],axis=1)
#df_tmp_tr_2 = pd.concat([df_1_tr,df_3_tr],axis=1)
#df_tmp_ts_2 = pd.concat([df_tmp_ts_1,df_3_ts],axis=1)
#df_tmp_ts_2 = pd.concat([df_1_ts,df_3_ts],axis=1)

In [71]:
#efficientnetv2b2,resnet152,jaginza(ginza_electra_text)
#df_tmp_tr_3 = pd.concat([df_tmp_tr_2,df_4_tr],axis=1)
#df_tmp_ts_3 = pd.concat([df_tmp_ts_2,df_4_ts],axis=1)

In [72]:
#train_tmp = df_tmp_tr_3.drop(["is_laugh","odai_photo_file_name"], axis=1)

In [73]:
#test_tmp = df_tmp_ts_3.drop(["odai_photo_file_name"], axis=1)

In [74]:
#df_5_tr = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_image_beit.csv")
#df_5_ts = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_image_beit.csv")

In [75]:
#df_5_tr = df_5_tr.drop(["Unnamed: 0","odai_photo_file_name","id", "text","is_laugh"], axis=1)
#df_5_ts = df_5_ts.drop(["Unnamed: 0","odai_photo_file_name","id", "text"], axis=1)

In [76]:
#train_tmp = pd.concat([train_emb,df_5_tr],axis=1)
#train_tmp = pd.concat([train_emb,df_4_tr],axis=1)
#test_tmp = pd.concat([test_emb,df_5_ts],axis=1)
#test_tmp = pd.concat([test_emb,df_4_ts],axis=1)

In [77]:
df_6_tr = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_rinna_tune_clip.csv")
df_6_ts = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_rinna_tune_clip.csv")

In [78]:
df_6_tr = df_6_tr.drop(["img_path","Unnamed: 0","odai_photo_file_name","id", "text","is_laugh"], axis=1)
df_6_ts = df_6_ts.drop(["img_path","Unnamed: 0","odai_photo_file_name","id", "text"], axis=1)

In [79]:
#df_6_tr = df_6_tr.drop(df_6_tr.columns[0:1030], axis=1)

In [80]:
#df_6_ts = df_6_ts.drop(df_6_ts.columns[0:1029], axis=1)

In [81]:
#train_tmp_2 = pd.concat([train_tmp,df_6_tr],axis=1)
#test_tmp_2 = pd.concat([test_tmp,df_6_ts],axis=1)

In [82]:
#cos_tr = pd.read_csv('/content/drive/MyDrive/nishika/output/tsne_cos_sim_tr_df.csv')
#cos_ts = pd.read_csv('/content/drive/MyDrive/nishika/output/tsne_cos_sim_ts_df.csv')

In [83]:
#train = pd.concat([train_tmp_2,train_df],axis=1)
#train = pd.concat([train_tmp,cos_tr],axis=1)
#test = pd.concat([test_tmp_2,test_df],axis=1)
#test = pd.concat([test_tmp,cos_ts],axis=1)

In [84]:
tr_debv3 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_debertav3.csv")
ts_debv3 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_debertav3.csv")

In [85]:
tr_debv3 = tr_debv3.drop(["Unnamed: 0","is_laugh","id","text","odai_photo_file_name"], axis=1)

In [86]:
ts_debv3 = ts_debv3.drop(["Unnamed: 0","id","text","odai_photo_file_name"], axis=1)

In [87]:
tr_debv2 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_debertav2.csv")
ts_debv2 = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_debertav2.csv")

In [88]:
tr_debv2 = tr_debv2.drop(["Unnamed: 0","is_laugh","id","text","odai_photo_file_name"], axis=1)
ts_debv2 = ts_debv2.drop(["Unnamed: 0","id","text","odai_photo_file_name"], axis=1)

In [89]:
tr_deb = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_deberta.csv")
ts_deb = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_deberta.csv")

In [90]:
tr_deb = tr_deb.drop(["Unnamed: 0","is_laugh","id","text","odai_photo_file_name"], axis=1)
ts_deb = ts_deb.drop(["Unnamed: 0","id","text","odai_photo_file_name"], axis=1)

In [91]:
tr_tmp_db1= pd.concat([tr_deb,tr_debv2],axis=1)
ts_tmp_db1 = pd.concat([ts_deb,ts_debv2],axis=1)

In [92]:
tr_tmp_db2= pd.concat([tr_tmp_db1,tr_debv3],axis=1)
ts_tmp_db2 = pd.concat([ts_tmp_db1,ts_debv3],axis=1)

In [93]:
train_tmp_2 = pd.concat([tr_tmp_db1,tr_tmp_db2],axis=1)
test_tmp_2 = pd.concat([ts_tmp_db1,ts_tmp_db2],axis=1)

In [94]:
#tr_tmpo = pd.concat([train_tmp,tr_deb],axis=1)
#ts_tmpo = pd.concat([test_tmp,ts_deb],axis=1)

In [95]:
train_tmpo = pd.concat([df_2_tr,df_4_tr],axis=1)
test_tmpo = pd.concat([df_2_ts,df_4_ts],axis=1)

In [96]:
train_tmpo = pd.concat([train_tmpo,df_6_tr],axis=1)
test_tmpo = pd.concat([test_tmpo,df_6_ts],axis=1)

In [97]:
train_tmp = pd.concat([train_tmpo,train_tmp_2],axis=1)
test_tmp = pd.concat([test_tmpo,test_tmp_2],axis=1)

In [98]:
tr_vit = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_train_image_timm_dm_vit_large_patch16_224.csv")
ts_vit = pd.read_csv("/content/drive/MyDrive/nishika/embeded/embedding_test_image_timm_dm_vit_large_patch16_224.csv")

In [99]:
tr_vit = tr_vit.drop(["Unnamed: 0","is_laugh","id","text","odai_photo_file_name"], axis=1)
ts_vit = ts_vit.drop(["Unnamed: 0","id","text","odai_photo_file_name"], axis=1)

In [100]:
tr_tmp_timm = pd.concat([tr_vit,train_tmp],axis=1)
ts_tmp_timm = pd.concat([ts_vit,test_tmp],axis=1)

In [101]:
train = pd.concat([tr_tmp_timm,train_df],axis=1)
test = pd.concat([ts_tmp_timm,test_df],axis=1)

In [102]:
print(train)

       vit_large_patch16_224_0  vit_large_patch16_224_1  \
0                    -0.174948                 1.216538   
1                     0.759446                 0.550291   
2                     0.445224                -0.187247   
3                    -0.329393                 0.633074   
4                     0.406402                 1.002488   
...                        ...                      ...   
24957                 1.024891                -0.834444   
24958                -0.776498                 0.833142   
24959                -0.751882                 1.186205   
24960                 0.502788                 2.710541   
24961                -1.214408                 1.231479   

       vit_large_patch16_224_2  vit_large_patch16_224_3  \
0                    -2.196662                 0.005932   
1                    -1.803098                -1.484083   
2                    -1.625515                -1.966000   
3                    -1.377089                -1.452802

In [103]:
train["text_len"] = train["text"].str.len()
test["text_len"] = test["text"].str.len()

In [104]:
y = train["is_laugh"]
X = train.drop(["odai_photo_file_name","is_laugh","id","text"], axis=1)

X_test = test.drop(["odai_photo_file_name","id","text"], axis=1)

In [105]:

X = X.T.drop_duplicates().T
X_test = X_test.T.drop_duplicates().T


# Model

In [108]:

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [109]:

params = {  
    "n_estimators": 20000,
    "objective": 'binary',
    "learning_rate": 0.01,
    'lambda_l1': 8.553323365580143, 
    'lambda_l2': 0.00024389221237366783, 
    'num_leaves': 240, 
    'feature_fraction': 0.6839999999999999, 
    'bagging_fraction': 0.6378550454605715, 
    'bagging_freq': 1,
    "random_state": 71,
    "n_jobs": -1,
    "importance_type": "gain",
    'colsample_bytree': .8,
    "reg_lambda": 5,
    "max_depth":9,
    'min_child_samples': 20
    }

In [111]:
# 予測値を格納するdf
# df to store the predicted value
preds_lgb = pd.DataFrame()

for k, (tr_id, vl_id) in enumerate(kf.split(X, y)):
    print("="*50)
    print(f"               KFold{k+1}")
    print("="*50)
    
    X_train, X_val = X.iloc[tr_id, :], X.iloc[vl_id, :]
    y_train, y_val = y.iloc[tr_id], y.iloc[vl_id]
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    
    model_lgb = lgb.train(params=params,
                          train_set=lgb_train,
                          valid_sets=lgb_val,
                          num_boost_round=100000,
                          early_stopping_rounds=200,
                          verbose_eval=1000)
    
    pred_lgb = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)
    pred_lgb = pd.DataFrame(pred_lgb)
    
    # 予測値を横に連結していく
    # Concatenate the predictions horizontally
    preds_lgb = pd.concat([preds_lgb, pred_lgb], axis=1)

               KFold1


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[670]	valid_0's binary_logloss: 0.653249
               KFold2
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[654]	valid_0's binary_logloss: 0.648399
               KFold3
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's binary_logloss: 0.642356
Early stopping, best iteration is:
[1157]	valid_0's binary_logloss: 0.642052
               KFold4
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[440]	valid_0's binary_logloss: 0.655888
               KFold5
Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's binary_logloss: 0.647599
Early stopping, best iteration is:
[873]	valid_0's binary_logloss: 0.646949


In [112]:
preds_lgb

,0,0,0,0,0
0,0.482554,0.446997,0.504518,0.491934,0.522277
1,0.275820,0.285046,0.316515,0.404659,0.308589
2,0.334267,0.333693,0.228902,0.304751,0.283562
3,0.420295,0.442489,0.366241,0.443399,0.366738
4,0.840843,0.819668,0.864066,0.785993,0.872279
...,...,...,...,...,...
5995,0.397065,0.423405,0.504543,0.360466,0.425254
5996,0.697869,0.759454,0.735121,0.749291,0.735248
5997,0.620300,0.574703,0.531786,0.532034,0.578922
5998,0.577228,0.494909,0.536858,0.542090,0.560758


In [113]:
label = preds_lgb.mean(axis=1)
label

0       0.489656
1       0.318126
2       0.297035
3       0.407832
4       0.836570
          ...   
5995    0.422147
5996    0.735397
5997    0.567549
5998    0.542369
5999    0.705815
Length: 6000, dtype: float64

In [114]:
submission_df["is_laugh"] = label

submission_df.head()

,id,is_laugh
0,rfdjcfsqq,0.489656
1,tsgqmfpef,0.318126
2,owjcthkz2,0.297035
3,rvgaocjyy,0.407832
4,uxtwu5i69,0.836570


In [115]:
submission_df.to_csv(('/content/drive/MyDrive/nishika/sub.csv'), index=False)